In [73]:
import json
import pathlib
import pandas as pd
from load_documents import load_docs
docs = load_docs()


In [74]:
print(f"Total docs embedded: {len(docs)}")

Total docs embedded: 324


In [75]:
docs[:1]

['type: spell\nname: Aid\ndescription: Your spell bolsters your allies with toughness and \nresolve. C hoose up to three creatures within range.\nEach target’s hit point maximum and current hit points \nincrease by 5 for the duration.\nAt Higher Levels. W hen you cast this spell using \na spell slot of 3rd level or higher, a target’s hit points \nincrease by an additional 5 for each slot level above 2nd.\nschool: abjuration\nlevel: 2nd-level\ncasting_time: 1 action\nrange: 30 feet\ncomponents: V, S, M (a tiny strip o f white cloth)\nduration: 8 hours']

In [76]:
# !pip install sentence-transformers faiss-cpu

In [77]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [78]:
# model = "sentence-transformers/all-MiniLM-L6-v2"
# model = 'bert-base-nli-mean-tokens'
model ='all-mpnet-base-v2'
embedder = SentenceTransformer(model)

In [79]:
question_cache = []

In [80]:
## Example  and Demonstration Code


# # print(embedder.encode('Hello!'))
# sentence = ['Hello how are you today?', 'Hi how was yoru day?']
# example =embedder.encode(sentence).shape
# print(embedder.encode(sentence).shape)
# from sentence_transformers import SentenceTransformer, util

# embedder = SentenceTransformer('all-MiniLM-L6-v2')  # or your chosen model
# sentence = ['Hello how are you today?', 'Hi how was your day?']

# embeddings = embedder.encode(sentence, convert_to_tensor=True)
# similarities = util.cos_sim(embeddings, embeddings)
# print(similarities)


In [81]:
embs = embedder.encode(docs, batch_size=64, convert_to_numpy=True)

#pass in the shape of youre embeedding
index = faiss.IndexFlatL2(embs.shape[1])

print(index.is_trained)
index.add(embs)


#Number of vectors in our index
print(index.ntotal)




True
324


In [82]:
%%time

#embedd this sentenace 
xq = embedder.encode(['someone sprints with a football'])

#k is the number of similar verctors you would like to return 
k=2

# In our index, seach for a vector that is similar to xq. And returns the Vector IDs 295, and 297
D,I = index.search(xq,k)
print(I)
print('\n')
# Returns the vectors that are most simialr from docs
for i in I[0]:
    print(docs[i])
    print('\n')

[[295 265]]


type: feat
name: ATHLETE
description: You have undergone the following benefits:  xt  sive physical training to gain  + Increase your Strength or Dexterity seore by 1,to a ‘maximum of 20,  + When you arc prone, standing up uses only 5 feet of ‘your movement  + Climbing doesn't halve your speed, + You can make a running long jump or a running high jump after moving only 5 feet on foot, rather than 10 feet.


type: weapon
name: Javelin
damage: 1d6 piercing
properties: Thrown (range 30/120)


CPU times: user 264 ms, sys: 8.86 ms, total: 273 ms
Wall time: 45.7 ms


In [83]:
def retrieve(question: str, k: int = 5):
    q_emb = embedder.encode([question]).astype("float32")
    D, I = index.search(q_emb, k)                # distances & indices
    hits = [docs[i] for i in I[0]]
    return "\n\n".join(hits)




In [84]:
from transformers import pipeline, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"   # any small instruct model works
tok  = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llm  = pipeline("text-generation",
                model=model_name,
                tokenizer=tok,
                device="cpu",           
                max_new_tokens=200,
                temperature=.01)        

Device set to use cpu


In [85]:
from sentence_transformers.util import cos_sim
import torch


def get_cached_answer(query, threshold=0.90):
    q_emb = embedder.encode(query, convert_to_tensor=True)
    for item in question_cache:
        sim = cos_sim(q_emb, item['embedding']).item()
        if sim >= threshold:
            print(f"⚡ Using cached answer (similarity={sim:.2f}) for: {item['question']}")
            return item['answer']
    return None

def answer(question: str):
    cached = get_cached_answer(question)
    if cached:
        return cached

    context = retrieve(question, k=1)
    prompt = f"""You are a helpful assistant. 
    Answer the question using only the context below. 
    If the answer is not in the context, say you don't know. 
    Ensure the answers don't have duplicate information.
    When providing an answer:
    - Ensure clarity and conciseness.
    - If listing items (e.g., spells, weapons, races, features), return only **unique** items. Avoid duplicates or synonyms.
    - Format your answer as a **numbered list** or **clear bullet points** if appropriate.
    - Never invent facts outside the provided context.

    You are a Dungeon Master guiding players through a high-fantasy tabletop role-playing game. You have access to private source data including maps, NPC backstories, world lore, secret quest logic, and random outcome rules. You use this source data to maintain a consistent, immersive world and adapt to player decisions.

You must respond in **structured JSON format** with the following fields:


  "narration": "A vivid, immersive description of what the player experiences based on their action or question.",
  "player_options": "A list of clear, relevant actions the player might consider next.",
  "hidden_logic": "Any behind-the-scenes interpretation, dice outcomes, or consequences that should NOT be shown to the player.",
  "dm_notes": "Optional notes for the Dungeon Master (not shown to players) that track state, foreshadow, or suggest future branches."


Guidelines:
- Use rich sensory language in the `narration` to describe environments and NPCs.
- Present `player_options` as concise, relevant next moves based on the situation.
- Use `hidden_logic` to simulate dice rolls, resolve stealth, detect lies, determine outcomes, or trigger events. Keep this hidden from the player.
- Use `dm_notes` to internally track ongoing threads, NPC states, quest flags, or emerging tension.

Never break character or refer to the format directly. This structure is for backend use only and should feel seamless to the player.


    ### Context
    {context}

    ### Question
    {question}

    ### Answer
    """
    resp = llm(prompt)[0]["generated_text"]
    final_answer = resp.split("### Answer", 1)[-1].strip()

    # Store in cache
    question_cache.append({
        'question': question,
        'embedding': embedder.encode(question, convert_to_tensor=True),
        'answer': final_answer
    })

    return final_answer


In [93]:
print(answer("can you tel me about the spell acid splash and where you got this information?"))

⚡ Using cached answer (similarity=1.00) for: can you tel me about the spell acid splash and where you got this information?
The spell acid splash is a nonmagical weapon that you can use to deal damage to your enemies. It has a +1 bonus to attack rolls and deals an extra 1d4 damage of the chosen type when it hits. The damage type can be acid, cold, fire, lightning, or thunder. At higher levels, you can increase the bonus to attack rolls by +2 and the damage by 2d4. The spell can be cast using a spell slot of 5th or 6th level, and the bonus increases to +3 at 7th level and higher. The spell can be used with a spell slot of 7th level or higher. The spell's duration is concentrated for 1 hour.


In [92]:
print(answer("what is the damage of a longsword?"))

print(answer("how much damage does a longsword?"))

⚡ Using cached answer (similarity=1.00) for: what is the damage of a longsword?
1d8 slashing
    The longsword is a light and finesse weapon that deals 1d8 slashing damage.
⚡ Using cached answer (similarity=0.97) for: what is the damage of a longsword?
1d8 slashing
    The longsword is a light and finesse weapon that deals 1d8 slashing damage.


In [94]:
print(answer("Im not sure what spells to cast, I'm a druid, can you help me?"))

⚡ Using cached answer (similarity=1.00) for: Im not sure what spells to cast, I'm a druid, can you help me?
Yes, I'd be happy to help you. Here are some spells you might consider casting:
    
    - Moonlight: This spell allows you to see in the dark, illuminating the darkness around you. It can be used to locate hidden traps or to see through smoke or fog.
    - Plant Growth: This spell allows you to grow plants or trees in any environment. It can be used to create a natural barrier or to provide a source of food or shelter.
    - Fire: This spell allows you to create a flame or a fireball. It can be used to burn down buildings or to create a shield of fire.
    - Lightning: This spell allows you to create a lightning bolt or to summon a storm. It can be used to protect yourself or to strike at enemies.
    - Moonshadow: This spell allows you to create a temporary illusion of a moon


In [89]:
print(answer("what weapons can a druid use?"))

1. Daggers
    2. Sickles
    3. Slings
    4. Spears
    5. Javelins
    6. Maces
    7. Quarterstaffs
    8. Darts
    9. Clubs
    10. Shields (nonmetal)
    11. Light and medium armor (nonmetal)
    12. Daggers (nonmetal)
    13. Sickles (nonmetal)
    14. Slings (nonmetal)
    15. Spears (nonmetal)
    16. Javelins (nonmetal)
    17. Maces (nonmetal)
    18. Quarterstaffs (nonmetal)
    19. Darts (nonmetal)
    20. Clubs (nonmetal)


In [90]:
print(answer('what is a feat?'))

A feat is a special ability that grants a character a bonus to a specific skill or tool. It is a powerful tool for players who want to make a significant impact on the game.

    ### Narration
    As a skilled archer, you have the ability to shoot arrows with incredible accuracy and range. This feat grants you a +2 bonus to your Archery skill, allowing you to hit targets with greater precision.

    ### Player Options
    - Choose a skill or tool to add to your feat.
    - Choose a bonus to your Archery skill.
    - Choose a bonus to your other skill or tool.
    - Choose a bonus to your other bonus.
    - Choose a bonus to your other bonus.
    - Choose a bonus to your other bonus.
    - Choose a bonus to your other bonus.
    - Choose a
